In [ ]:
import os
path = '../work/sgdml/cspbbr3/cspbbr3-hightemp-run/'
with open('folders.txt', 'r') as f:
    folders = f.readlines()
    folders = [path + f.strip() for f in folders]

for folder in folders:
    try:
        for x in os.listdir(folder):
            if '.pbs' in x:
                pbs_file = x
    except FileNotFoundError:
        continue

    with open(f'{folder}/{pbs_file}', 'r') as f:
        python_line = f.readlines()[18]
        temp_arg = python_line.split()[6]
        temp = int(temp_arg.split('=')[1])
        im = 0
    for file in os.listdir(f'{folder}/snapshots'):
        i = int(file.split('-')[1].split('.')[0])
        if i > im:
            im = i
    print(f'max file index: {im}')
    all_snapshots = ''
    for i in range(im + 1):
        with open(f'{folder}/snapshots/cspbbr3-{i}.xyz', 'r') as f:
            all_snapshots += f.read()
    with open(f'datasets/40-cspbbr3-{temp}K.xyz', 'w') as f:
        f.write(all_snapshots)

In [1]:
import os
from torch.utils.data import Dataset
import torch
import pandas as pd
import numpy as np
from ase import io
from tqdm import tqdm


class xyzDataset(Dataset):
    def __init__(self, df, target='energy'):
        """
        :type df: DataFrame
        """
        x_col, y_col = [], []
        for col in df.columns: 
            if 'position' in col:
                x_col.append(col)
        if target == 'energy':
            y_col.append('E')
        elif target == 'force':
            for col in df.columns:
                if 'force' in col:
                    y_col.append(col)
        self.df = df
        self.x_col = x_col
        self.y_col = y_col
        self.n_samples = len(df)

    def __getitem__(self, index):
        x = torch.Tensor(self.df.loc[index, self.x_col].values)
        y = torch.Tensor(self.df.loc[index, self.y_col].values)
        return x, y

    def __len__(self):
        return self.n_samples

def atoms_dict(atoms):
    d = {}
    for i, (s, pos) in enumerate(zip(atoms.symbols, atoms.positions)):
        d[str(i + 1) + s + '_position_x'] = pos[0]
        d[str(i + 1) + s + '_position_y'] = pos[1]
        d[str(i + 1) + s + '_position_z'] = pos[2]
    try:
        for i, (s, f) in enumerate(zip(atoms.symbols, atoms.get_forces())):
            d[str(i + 1) + s + '_force_x'] = f[0]
            d[str(i + 1) + s + '_force_y'] = f[1]
            d[str(i + 1) + s + '_force_z'] = f[2]
    except RuntimeError:
        pass
    return d

def dataframe_from_xyz(xyz_file):
    molecules = io.read(xyz_file, index=':')
    molecules = [mol for mol in molecules if mol.get_calculator() is not None] # filter incomplete outputs from trajectory
    lattice = np.array(molecules[0].get_cell())
    Z = np.array([mol.get_atomic_numbers() for mol in molecules])
    all_z_the_same = (Z == Z[0]).all()
    if not all_z_the_same:
        print('Order of atoms changes accross dataset!\n Bye!')
        exit()
    z = Z[0]
    df = pd.DataFrame()
    for mol in molecules:
        line_dict = dict(E=mol.get_potential_energy())
        line_dict.update(atoms_dict(mol))
        df = df.append(line_dict, ignore_index=True)
    return df

In [2]:
df = dataframe_from_xyz('datasets/40-cspbbr3-1016K.xyz')

In [7]:
from torch import nn
import torch.functional as F


class energy_nn(nn.Module):
    def __init__(self, input_size, hidden1_size=128, hidden2_size=128):
        super(energy_nn, self).__init__()
        self.dense1 = nn.Linear(input_size, hidden1_size)
        self.active1 = nn.ReLU()
        self.dense2 = nn.Linear(hidden1_size, hidden2_size)
        self.active2 = nn.ReLU()
        self.dense_out = nn.Linear(hidden2_size, 1)

    def forward(self, inputs):
        x = self.active1(self.dense1(inputs))
        x = self.active2(self.dense2(x))
        x = self.dense_out(x)
        return x


In [16]:
from data_utils import *
from model import *
from scipy import stats
from sklearn.metrics import mean_squared_error
from ase.io import read
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
import petname

name = petname.Generate(3, ' ', 10)
print(f'starting: {name}')
writer = SummaryWriter(log_dir=f'runs/{name}')

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Let\'s use {device}!')

n_atoms = read('40-cspbbr3-795K.xyz').get_global_number_of_atoms()

EPOCHS = 100
batch_size = 100
lr = 1e-4
RELOAD_DATA = True

training_datasets = ['datasets/40-cspbbr3-355K.xyz']
if RELOAD_DATA:
    df_train = pd.DataFrame()
    for td in tqdm(training_datasets):
        df_train = df_train.append(dataframe_from_xyz(td), ignore_index=True)
    df_train.to_json('df_train.json')
else:
    df_train = pd.read_json('df_train.json')

starting: nicely clear ape
Let's use cuda!


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:48<00:00, 48.89s/it]


In [17]:
# df_train = dataframe_from_xyz('datasets/40-cspbbr3-795K.xyz')
trainset = torch.utils.data.DataLoader(xyzDataset(df_train, target='energy'), batch_size=batch_size, shuffle=True)
df_test = dataframe_from_xyz('datasets/40-cspbbr3-300K.xyz')
testset = torch.utils.data.DataLoader(xyzDataset(df_test, target='energy'), batch_size=batch_size, shuffle=True)

In [18]:
net = energy_nn(n_atoms * 3, hidden1_size=128, hidden2_size=128)
optimizer = optim.Adam(net.parameters(), lr=lr)
criteria = nn.SmoothL1Loss()  # reduction='sum'
net.to(device)

energy_nn(
  (dense1): Linear(in_features=120, out_features=128, bias=True)
  (active1): ReLU()
  (dense2): Linear(in_features=128, out_features=128, bias=True)
  (active2): ReLU()
  (dense_out): Linear(in_features=128, out_features=1, bias=True)
)

In [19]:
X = torch.rand(1, n_atoms * 3).to(device)
writer.add_graph(net, X)

In [20]:
def test_loss(nn, testset, criteria):
    with torch.no_grad():
        for i, (X, y) in enumerate(testset):
            X = X.to(device=device).float()
            y = y.to(device=device).float()
            output = nn(X)
            y = y.view(-1, 1)
            val_loss = criteria(y, output)
            break
        return val_loss


def test_model(nn, testset):
    with torch.no_grad():
        for i, (X, y) in enumerate(testset):
            X = X.to(device=device).float()
            y = y.to(device=device).float()
            output = nn(X)
            y = y.view(-1, 1)
            break
        x = [i[0].to(device=torch.device('cpu')).detach().numpy() for i in output]
        t = [i[0].to(device=torch.device('cpu')).detach().numpy() for i in y]
        RMSE = mean_squared_error(x, t, squared=False)
        MSE = mean_squared_error(x, t, squared=True)
        a, b, r2 = get_linregress(x, t)
        return x, t, r2, a, b, RMSE, MSE


def get_linregress(x, y):
    with torch.no_grad():
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
        if np.isnan(r_value):
            r_value = 0
        if np.isnan(slope):
            slope = 0
        return slope, intercept, r_value ** 2

In [21]:
## Train!
n_iter = 0
for epoch in tqdm(range(EPOCHS)):
    for i, (X, y) in enumerate(trainset):
        X = X.to(device=device)
        y = y.to(device=device)
        
        output = net(X)
        y = y.view(-1, 1)
        
        
        loss = criteria(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        val_loss = test_loss(net, testset, criteria)
        writer.add_scalar('Loss/train', loss, n_iter)
        writer.add_scalar('Loss/val_loss', val_loss, n_iter)
        x, t, r2, a, b, RMSE, MSE = test_model(net, testset)
        writer.add_scalar('Regression/slope', a, n_iter)
        writer.add_scalar('Regression/intercept', b, n_iter)
        writer.add_scalar('Regression/R2', r2, n_iter)
        writer.add_scalar('Regression/RMSE', RMSE, n_iter)
        writer.add_scalar('Regression/MSE', RMSE, n_iter)
        n_iter += 1
    val_loss = test_loss(net, testset, criteria)
    print(f'validation los: {val_loss}')

  1%|█▍                                                                                                                                                | 1/100 [00:26<44:01, 26.68s/it]

validation los: 131.97091674804688


  2%|██▉                                                                                                                                               | 2/100 [00:53<44:07, 27.01s/it]

validation los: 94.99209594726562


  3%|████▍                                                                                                                                             | 3/100 [01:20<43:02, 26.62s/it]

...

validation los: 0.06354916095733643


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 99/100 [39:21<00:21, 21.46s/it]

validation los: 0.051377832889556885


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [39:42<00:00, 23.83s/it]

validation los: 0.05585085600614548
